In [1]:
import numpy as np

import tensorflow as tf
import keras
import pandas as pd
try:
    from tqdm import tqdm
except:
    !pip install tqdm
    from tqdm import tqdm
import numpy as np
# from sklearn.decomposition import IncrementalPCA
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
sns.set(rc={'figure.figsize':(20,10)})
pd.set_option('display.max_columns', 100)
sns.set(font_scale=1.5)

Using TensorFlow backend.


  Using cached https://files.pythonhosted.org/packages/6c/4b/c38b5144cf167c4f52288517436ccafefe9dc01b8d1c190e18a6b154cd4a/tqdm-4.31.1-py2.py3-none-any.whl
You are using pip version 10.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import os
data_dir = "el_train/"
filenames = [[data_dir+x] for x in os.listdir(data_dir) if "csv" in x]

In [3]:
len(filenames)

2400

In [4]:
len(pd.read_csv(filenames[0][0], nrows=1).columns)

75

In [5]:
# ['memshpno', 'BrndMon1', 'BrndMon2', 'BrndMon3', 'BrndMon4', 'BrndMon5', 'BrndMon6', 'BrndMon7', 'BrndMon8', 'BrndMon9', 'BrndMon10', 'BrndMon11', 'BrndMon12', 'CompMon1', 'CompMon2', 'CompMon3', 'CompMon4', 'CompMon5', 'CompMon6', 'CompMon7', 'CompMon8', 'CompMon9', 'CompMon10', 'CompMon11', 'CompMon12', 'Num_of_unique_brands_bought_qtr_1', 'Total_num_of_receipts_qtr_1', 'Num_of_unique_products_bought_qtr_1', 'Total_amount_spent_qtr_1', 'Num_of_times_availed_discount_qtr_1', 'Time_since_first_purchase_qtr_1', 'Time_since_last_purchase_qtr_1', 'Num_of_unique_brands_bought_qtr_2', 'Total_num_of_receipts_qtr_2', 'Num_of_unique_products_bought_qtr_2', 'Total_amount_spent_qtr_2', 'Num_of_times_availed_discount_qtr_2', 'Time_since_first_purchase_qtr_2', 'Time_since_last_purchase_qtr_2', 'Num_of_unique_brands_bought_qtr_3', 'Total_num_of_receipts_qtr_3', 'Num_of_unique_products_bought_qtr_3', 'Total_amount_spent_qtr_3', 'Num_of_times_availed_discount_qtr_3',
# 'Time_since_first_purchase_qtr_3', 'Time_since_last_purchase_qtr_3', 'Num_of_unique_brands_bought_qtr_4', 'Total_num_of_receipts_qtr_4', 'Num_of_unique_products_bought_qtr_4', 'Total_amount_spent_qtr_4', 'Num_of_times_availed_discount_qtr_4', 'Time_since_first_purchase_qtr_4', 'Time_since_last_purchase_qtr_4', 'target_month', 'label']

In [6]:
cols_to_read = list(pd.read_csv(filenames[0][0], nrows=1).columns)
columnsToUse = list(pd.read_csv(filenames[0][0], nrows=1).columns)

columnsToUse.pop(columnsToUse.index("memshpno"))
columnsToUse.pop(columnsToUse.index("target_month"))

labelColumn = "label"
recordDefaults = [[0.0] for col in columnsToUse]

select_cols = [i for i,x in enumerate(pd.read_csv(filenames[0][0], nrows=1).columns) if x in columnsToUse]

In [7]:
cols = np.array(columnsToUse[:-1])

In [8]:
cols.shape

(72,)

In [9]:
cols

array(['el_products_bought_mon1', 'unique_receipts_mon1',
       'unique_products_bought_mon1', 'amount_spent_mon1',
       'discount_on_beauty_products_mon1', 'times_availed_discount_mon1',
       'el_products_bought_mon2', 'unique_receipts_mon2',
       'unique_products_bought_mon2', 'amount_spent_mon2',
       'discount_on_beauty_products_mon2', 'times_availed_discount_mon2',
       'el_products_bought_mon3', 'unique_receipts_mon3',
       'unique_products_bought_mon3', 'amount_spent_mon3',
       'discount_on_beauty_products_mon3', 'times_availed_discount_mon3',
       'el_products_bought_mon4', 'unique_receipts_mon4',
       'unique_products_bought_mon4', 'amount_spent_mon4',
       'discount_on_beauty_products_mon4', 'times_availed_discount_mon4',
       'el_products_bought_mon5', 'unique_receipts_mon5',
       'unique_products_bought_mon5', 'amount_spent_mon5',
       'discount_on_beauty_products_mon5', 'times_availed_discount_mon5',
       'el_products_bought_mon6', 'unique_rec

In [10]:
# Timeseries
# Aggregate to month
# Category Specifc/Brand Specific

# Try both Category

In [11]:
b_size = 1024

total_examples = 442553

test_examples = total_examples//10

test_examples

44255

In [12]:
def read_dataset(filenames, b_size=1024, split=10000):
    def decode_csv(value_column):
        columns = tf.decode_csv(value_column, record_defaults=recordDefaults, select_cols=select_cols)
        ts = columns[:-1]
        label = columns[-1]
        return ts, label

    # Create dataset from file list
    dataset = tf.data.Dataset.from_tensor_slices(filenames)
    dataset = dataset.flat_map(lambda filename: tf.data.TextLineDataset(filename).skip(1))
    dataset = dataset.map(decode_csv)  # Transform each elem by applying decode_csv fn

    if True:
        num_epochs = None # indefinitely
    else:
        num_epochs = 1 # end-of-input after this

    dataset = dataset.shuffle(10240)
    test_data = dataset.take(split)
    train_data = dataset.skip(split)

    test_data = test_data.repeat(num_epochs).batch(b_size)
    train_data = train_data.repeat(num_epochs).batch(b_size)

    test_iterator = test_data.make_one_shot_iterator()
    train_iterator = train_data.make_one_shot_iterator()

    def make_gen_from_iterator(osi):
        sess = tf.Session()
        next_val = osi.get_next()
        while True:
            with sess.as_default():
                ts, labels = sess.run(next_val)
                ts = np.reshape(ts, newshape=(-1,12,6))
                yield ts, labels

    test_gen = make_gen_from_iterator(test_iterator)
    train_gen = make_gen_from_iterator(train_iterator)
    
    return train_gen, test_gen

In [13]:
train_gen, test_gen = read_dataset(filenames=filenames, b_size=b_size, split=test_examples)

In [14]:
temp = train_gen.next()

In [15]:
temp[0].shape

(1024, 12, 6)

In [16]:
temp[1].shape

(1024,)

In [17]:
_ = test_gen.next()

## Get Train Set Size

In [18]:
train_set_size = total_examples - total_examples//10

In [19]:
train_steps_per_epoch = train_set_size//b_size
train_steps_per_epoch

388

## Get Test Set Size

In [20]:
test_set_size = total_examples//10

In [21]:
test_steps_per_epoch = test_set_size//b_size
test_steps_per_epoch

43

## Creating the recurrent neural network model

In [22]:
# importing relevant libraries
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot

import tensorflow as tf
from tensorflow.python.keras import Model
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input, Dense ,GRU, Embedding, Flatten, RepeatVector, TimeDistributed, Dropout, Bidirectional, Reshape, concatenate
from tensorflow.python.keras.optimizers import RMSprop, Adam, Nadam
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.python.keras.initializers import RandomUniform
from tensorflow.python.keras import regularizers

We'll use the Keras API for its simplicity. The type of recurrent neural network we are using is called a gated recurrent unit (GRU). It will have 300 outputs for each timestep. The simple GRU will be wrapped inside a Bidirectional layer. Bidirectional layer creates two GRUs, in which one GRU traverses the signal in reverse order. Both of these GRUs will output 300 values at each timestep which are summed together at the end because we have set "merge_mode = True".

The last layer would be a dense layer with a "linear" activation function because we want to predict a continuous range of values.

In [23]:
def get_ts_model():
    # Creating the Optimizer
    #optimizer = Adam(lr=1e-2)
    optimizer = Adam()

    ts_input = Input(shape=(12,6), dtype='float32', name='main_input')

    # A LSTM will transform the vector sequence into a single vector,
    # containing information about the entire sequence
    gru_1 = Bidirectional(GRU(64, return_sequences = True))(ts_input)
    gru_2 = Bidirectional(GRU(64, return_sequences = True))(gru_1)
    gru_flattened_ts = Flatten()(gru_2)
    
    x = Dense(8, activation='relu')(gru_flattened_ts)
#     x = Dense(8, activation='relu')(x)

    # And finally we add the main logistic regression layer
    output = Dense(1, activation='sigmoid', name='output')(x)

    model = Model(inputs=[ts_input], outputs=[output])
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])
    
    return model

## Callback functions

Callback functions are creating for logging the progress of training in Tensorboard and saving the best performing models.

In [25]:
# defining the Checkpoint File name and path
path_checkpoint = 'ckpt_model_ts.keras'

callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                      monitor='val_loss',
                                      verbose=1,
                                      save_weights_only=True,
                                      save_best_only=True)

# Callback for early stopping i-e if the model starts to overfit
callback_early_stopping = EarlyStopping(monitor='val_acc',
                                        patience=4, verbose=1)

# Callback to reduce learning rate if the learning stangnates
callback_reduce_lr = ReduceLROnPlateau(monitor='val_acc',
                                       factor=0.1,
                                       min_lr=1e-4,
                                       patience=2,
                                       verbose=1)

# Callback for logging prgress in tensorboard
callback_tensorboard = TensorBoard(log_dir='./model/log/',
                                   histogram_freq=0,
                                   write_graph=True)

callbacks = [
#     callback_early_stopping,
             callback_checkpoint,
             callback_reduce_lr,
             callback_tensorboard]

## Model Training

In [26]:
class_weights = {0: 1.,
                1: 2.}

In [27]:
# model = get_model()

In [28]:
model = get_ts_model()

Instructions for updating:
Colocations handled automatically by placer.


In [29]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 12, 6)             0         
_________________________________________________________________
bidirectional (Bidirectional (None, 12, 128)           27264     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 12, 128)           74112     
_________________________________________________________________
flatten (Flatten)            (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 8)                 12296     
_________________________________________________________________
output (Dense)               (None, 1)                 9         
Total params: 113,681
Trainable params: 113,681
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit_generator(generator=train_gen,
                    epochs=10,
#                     run_tensorboard_locally=True,
                    steps_per_epoch=train_steps_per_epoch,
#                     class_weight=class_weights,
                    validation_data=test_gen,
                    validation_steps=test_steps_per_epoch, 
                    callbacks = callbacks,
                    verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
43/43 [==============================] - 15s 346ms/step - loss: 0.3052 - acc: 0.8952

Epoch 00001: val_loss improved from inf to 0.30518, saving model to ckpt_model_ts.keras
388/388 [==============================] - 227s 585ms/step - loss: 0.3159 - acc: 0.8895 - val_loss: 0.3052 - val_acc: 0.8952
Epoch 2/10
43/43 [==============================] - 45s 1s/step - loss: 0.3043 - acc: 0.8958

Epoch 00002: val_loss improved from 0.30518 to 0.30427, saving model to ckpt_model_ts.keras
388/388 [==============================] - 299s 771ms/step - loss: 0.3071 - acc: 0.8955 - val_loss: 0.3043 - val_acc: 0.8958
Epoch 3/10
43/43 [==============================] - 62s 1s/step - loss: 0.3032 - acc: 0.8968

Epoch 00003: val_loss improved from 0.30427 to 0.30319, saving model to ckpt_model_ts.keras
388/388 [==============================] - 887s 2s/step - loss: 0.3061 - acc: 0.8957 - val_loss: 0.3032 - val_acc: 0.8968
Epoch 4/10
43/43 [=====

In [33]:
# Validation producing Confusion Variables
tp, tn, fp, fn = 0,0,0,0
tot_true = 0

for i in tqdm(range(test_steps_per_epoch)):
    batch_in, y_true = test_gen.next()

    y_true = y_true.reshape((b_size,1))
    tot_true+=y_true.sum()
    y_pred = model.predict_on_batch(batch_in)
    y_pred_lab = (y_pred >= 0.5).astype("int")
    
    corr = (y_pred_lab == y_true.reshape(y_pred_lab.shape))

    tp += y_true[corr].sum()
    tn += y_true[corr].size - y_true[corr].sum()

    fn += y_true[~corr].sum()
    fp += y_true[~corr].size - y_true[~corr].sum()
    

tot_neg = (b_size*test_steps_per_epoch) - tot_true

100%|██████████| 43/43 [00:12<00:00,  3.43it/s]


### 1:1 Weights 5 Epochs

In [34]:
print(tn, fp, tot_neg)
print(fn, tp, tot_true)

(38687.0, 288.0, 38975.0)
(4280.0, 777.0, 5057.0)


In [35]:
pos_rec = tp/(tp+fn)# Recall
pos_prc = tp/(tp+fp)# Precision

print("Positive Class Precision: {}".format(pos_prc))
print("Positive Class Recall   : {}".format(pos_rec))

Positive Class Precision: 0.729577464789
Positive Class Recall   : 0.153648408147


In [36]:
neg_rec = tn/(tn+fp)# Recall
neg_prc = tn/(tn+fn)# Precision

print("Negative Class Precision: {}".format(neg_prc))
print("Negative Class Recall   : {}".format(neg_rec))

Negative Class Precision: 0.900388670375
Negative Class Recall   : 0.992610647851


In [37]:
model.save_weights("model_el_10_epochs.keras")

## Validation

In [35]:
data_dir = "m3_data/el_val/"
val_filenames = [[data_dir+x] for x in os.listdir(data_dir) if "csv" in x]

In [36]:
cols_to_read = list(pd.read_csv(filenames[0][0], nrows=1).columns)
columnsToUse = list(pd.read_csv(filenames[0][0], nrows=1).columns)

columnsToUse.pop(columnsToUse.index("memshpno"))
columnsToUse.pop(columnsToUse.index("target_month"))

labelColumn = "label"
recordDefaults = [[0.0] for col in columnsToUse]

select_cols = [i for i,x in enumerate(pd.read_csv(filenames[0][0], nrows=1).columns) if x in columnsToUse]

In [42]:
b_size = 1024

set_size = 39588

In [38]:
def read_validation_dataset(filenames, b_size=1024):
    def decode_csv(value_column):
        columns = tf.decode_csv(value_column, record_defaults=recordDefaults, select_cols=select_cols)
        ts = columns[:-1]
        label = columns[-1]
        return ts, label

    # Create dataset from file list
    dataset = tf.data.Dataset.from_tensor_slices(filenames)
    dataset = dataset.flat_map(lambda filename: tf.data.TextLineDataset(filename).skip(1))
    dataset = dataset.map(decode_csv)  # Transform each elem by applying decode_csv fn

    if True:
        num_epochs = None # indefinitely
    else:
        num_epochs = 1 # end-of-input after this

    test_data = dataset.shuffle(10000)

    test_data = test_data.repeat(num_epochs).batch(b_size)

    test_iterator = test_data.make_one_shot_iterator()

    def make_gen_from_iterator(osi):
        sess = tf.Session()
        next_val = osi.get_next()
        while True:
            with sess.as_default():
                ts, labels = sess.run(next_val)
                ts = np.reshape(ts, newshape=(-1,12,6))
                yield ts, labels

    test_gen = make_gen_from_iterator(test_iterator)
    
    return test_gen

In [39]:
val_gen = read_validation_dataset(filenames=val_filenames, b_size=b_size)

In [40]:
temp = val_gen.next()

In [43]:
val_steps_per_epoch = set_size//b_size
val_steps_per_epoch

38

In [7]:
model = get_ts_model()
model.load_weights("model_el_10_epochs.keras")

In [45]:
# Validation producing Confusion Variables
tp, tn, fp, fn = 0,0,0,0
tot_true = 0

for i in tqdm(range(val_steps_per_epoch)):
    batch_in, y_true = val_gen.next()

    y_true = y_true.reshape((b_size,1))
    tot_true+=y_true.sum()
    y_pred = model.predict_on_batch(batch_in)
    y_pred_lab = (y_pred >= 0.5).astype("int")
    
    corr = (y_pred_lab == y_true.reshape(y_pred_lab.shape))

    tp += y_true[corr].sum()
    tn += y_true[corr].size - y_true[corr].sum()

    fn += y_true[~corr].sum()
    fp += y_true[~corr].size - y_true[~corr].sum()
    
tot_neg = (b_size*val_steps_per_epoch) - tot_true

100%|██████████| 38/38 [00:12<00:00,  2.75it/s]


In [46]:
print(tn, fp, tot_neg)
print(fn, tp, tot_true)

(32265.0, 198.0, 32463.0)
(5806.0, 643.0, 6449.0)


In [47]:
print("Positive Predictions: {}".format(fp+tp))

Positive Predictions: 841.0


In [48]:
pos_rec = tp/(tp+fn)# Recall
pos_prc = tp/(tp+fp)# Precision

print("Positive Class Precision: {}".format(pos_prc))
print("Positive Class Recall   : {}".format(pos_rec))

Positive Class Precision: 0.764565992866
Positive Class Recall   : 0.0997053806792


In [49]:
neg_rec = tn/(tn+fp)# Recall
neg_prc = tn/(tn+fn)# Precision

print("Negative Class Precision: {}".format(neg_prc))
print("Negative Class Recall   : {}".format(neg_rec))

Negative Class Precision: 0.847495468992
Negative Class Recall   : 0.993900748544


# Generating and Saving Results

In [40]:
data_dir = "m3_data/el_val/"
val_filenames = [[data_dir+x] for x in os.listdir(data_dir) if "csv" in x]

In [41]:
cols_to_read = list(pd.read_csv(filenames[0][0], nrows=1).columns)
columnsToUse = list(pd.read_csv(filenames[0][0], nrows=1).columns)

# columnsToUse.pop(columnsToUse.index("target_month"))

labelColumn = "label"
recordDefaults = [[0.0] for col in columnsToUse]
recordDefaults[0] = [""]
select_cols = [i for i,x in enumerate(pd.read_csv(filenames[0][0], nrows=1).columns) if x in columnsToUse]

def read_dataset_w_memshpno(filenames, b_size=1024):
    def decode_csv(value_column):
        columns = tf.decode_csv(value_column, record_defaults=recordDefaults, select_cols=select_cols)
        memshpno = columns[0]
        ts = columns[1:-2]
        month = columns[-2]
        label = columns[-1]
        return memshpno, ts, month, label

    # Create dataset from file list
    dataset = tf.data.Dataset.from_tensor_slices(filenames)
    dataset = dataset.flat_map(lambda filename: tf.data.TextLineDataset(filename).skip(1))
    dataset = dataset.map(decode_csv)  # Transform each elem by applying decode_csv fn

    if True:
        num_epochs = None # indefinitely
    else:
        num_epochs = 1 # end-of-input after this

    test_data = dataset.shuffle(10000)

    test_data = test_data.repeat(num_epochs).batch(b_size)

    test_iterator = test_data.make_one_shot_iterator()

    def make_gen_from_iterator(osi):
        sess = tf.Session()
        next_val = osi.get_next()
        while True:
            with sess.as_default():
                memshpno, ts, month, labels = sess.run(next_val)
                ts = np.reshape(ts, newshape=(b_size,12,6))
                yield memshpno, ts, month, labels

    test_gen = make_gen_from_iterator(test_iterator)
    
    return test_gen

In [42]:
val_filenames

[['m3_data/el_val/part-02545-4daef64f-d0c0-4576-81fc-a8b0c319b0ac-c000.csv'],
 ['m3_data/el_val/part-02788-4daef64f-d0c0-4576-81fc-a8b0c319b0ac-c000.csv'],
 ['m3_data/el_val/part-02778-4daef64f-d0c0-4576-81fc-a8b0c319b0ac-c000.csv'],
 ['m3_data/el_val/part-02743-4daef64f-d0c0-4576-81fc-a8b0c319b0ac-c000.csv'],
 ['m3_data/el_val/part-02588-4daef64f-d0c0-4576-81fc-a8b0c319b0ac-c000.csv'],
 ['m3_data/el_val/part-02424-4daef64f-d0c0-4576-81fc-a8b0c319b0ac-c000.csv'],
 ['m3_data/el_val/part-02691-4daef64f-d0c0-4576-81fc-a8b0c319b0ac-c000.csv'],
 ['m3_data/el_val/part-02425-4daef64f-d0c0-4576-81fc-a8b0c319b0ac-c000.csv'],
 ['m3_data/el_val/part-02689-4daef64f-d0c0-4576-81fc-a8b0c319b0ac-c000.csv'],
 ['m3_data/el_val/part-02749-4daef64f-d0c0-4576-81fc-a8b0c319b0ac-c000.csv'],
 ['m3_data/el_val/part-02747-4daef64f-d0c0-4576-81fc-a8b0c319b0ac-c000.csv'],
 ['m3_data/el_val/part-02754-4daef64f-d0c0-4576-81fc-a8b0c319b0ac-c000.csv'],
 ['m3_data/el_val/part-02493-4daef64f-d0c0-4576-81fc-a8b0c319b0a

In [43]:
set_size = 0
for f in val_filenames:
    set_size += pd.read_csv(f[0]).shape[0]

In [44]:
set_size

39588

In [46]:
for i in range(1,1000):
    if set_size%i==0:
        b_size = i

In [47]:
b_size

12

In [48]:
# b_size = 12
# set_size = 39588
val_steps_per_epoch = set_size//b_size
val_steps_per_epoch

3299

In [57]:
data_iterator = read_dataset_w_memshpno(val_filenames, b_size=b_size)

In [61]:
model = get_ts_model()
model.load_weights("model_el_10_epochs.keras")

In [65]:
for i in tqdm(range(val_steps_per_epoch)):
    memshpnos, batch_in, month, y_true = data_iterator.next()
    y_pred = model.predict_on_batch(batch_in)
    y_pred = np.reshape(y_pred, (b_size,))
    
    batch_df = pd.DataFrame(data=np.column_stack((memshpnos, y_true, y_pred, month)), columns=("memshpno", "y_true", "y_pred", "month"))
    
    batch_df["jan"] = (batch_df["month"]==25).astype(int)
    batch_df["feb"] = (batch_df["month"]==26).astype(int)

    batch_df.drop(["month"], axis=1, inplace=True)
    
    with open("final_m3_el_4.csv", "a") as m3_file:
        batch_df.to_csv(m3_file, index=False, header=True if i==0 else False)

100%|██████████| 3299/3299 [00:52<00:00, 62.63it/s]


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("final_m3_el_4.csv")

In [3]:
df.shape

(39588, 5)

In [4]:
df.head()

,memshpno,y_true,y_pred,jan,feb
0,9.200006e+11,0.0,0.055884,0,1
1,6.400000e+11,0.0,0.057138,0,1
2,1.900001e+11,0.0,0.083713,1,0
3,9.400003e+11,1.0,0.254973,1,0
4,2.800000e+11,0.0,0.144156,0,1


In [71]:
bought_df = df[df['y_true']==1]

In [72]:
bought_df.head()

,memshpno,y_true,y_pred,jan,feb
3,9.400003e+11,1.0,0.254973,1,0
5,9.200008e+11,1.0,0.142765,1,0
7,9.200000e+11,1.0,0.340507,1,0
9,9.200003e+11,1.0,0.233682,0,1
10,9.200004e+11,1.0,0.104613,1,0


In [77]:
bought_df.shape

(6553, 5)

In [80]:
bought_df.drop_duplicates("memshpno").shape

(5302, 5)

In [76]:
bought_df[bought_df["y_pred"]>0.5].shape

(650, 5)

In [82]:
bought_df.drop(["y_true", "y_pred"], axis=1).to_csv("month_labels.csv", header=True, index=False)